<a href="https://colab.research.google.com/github/LucasWanderleyGomes/Projeto-de-Transfer-Learning/blob/main/Projeto_de_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparar ambiente

In [ ]:
# Instalar dependências
!pip install tensorflow tensorflow-datasets

## Importar bibliotecas principais

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

## Carregar o Dataset

In [ ]:
# Baixar o dataset
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True
)

# Verificar info
print(metadata)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.Y6MJRR_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings 

## Visualizar algumas imagens

In [ ]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(3):
    plt.imshow(image)
    plt.title(get_label_name(label))
    plt.axis("off")
    plt.show()

## Pré-processamento das imagens

In [ ]:
IMG_SIZE = 224

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

# Batch e shuffle
BATCH_SIZE = 32
train = train.shuffle(1000).batch(BATCH_SIZE).prefetch(1)
validation = validation.batch(BATCH_SIZE).prefetch(1)
test = test.batch(BATCH_SIZE).prefetch(1)


## Modelo Base

In [ ]:
# Carregar MobileNetV2 pré-treinada no ImageNet, sem a parte final
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Congelar camadas da base
base_model.trainable = False

# Adicionar camada final
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')  # binária: gatos vs cachorros
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Treinamento:

In [ ]:

EPOCHS = 5

history = model.fit(
    train,
    validation_data=validation,
    epochs=EPOCHS
)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 974s 2s/step - accuracy: 0.9450 - loss: 0.1418 - val_accuracy: 0.9815 - val_loss: 0.0466
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 888s 2s/step - accuracy: 0.9833 - loss: 0.0466 - val_accuracy: 0.9828 - val_loss: 0.0459
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 878s 2s/step - accuracy: 0.9855 - loss: 0.0393 - val_accuracy: 0.9819 - val_loss: 0.0517
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 930s 2s/step - accuracy: 0.9878 - loss: 0.0364 - val_accuracy: 0.9819 - val_loss: 0.0457
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 899s 2s/step - accuracy: 0.9886 - loss: 0.0346 - val_accuracy: 0.9824 - val_loss: 0.0468


## Avaliação no Test Set

In [ ]:
loss, accuracy = model.evaluate(test)
print(f"Acurácia no teste: {accuracy*100:.2f}%")

73/73 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.9869 - loss: 0.0337
Acurácia no teste: 98.71%


## Testar com Imagens Novas

In [ ]:
import numpy as np
from PIL import Image

def predict_image(path):
    img = Image.open(path).resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]
    label = "Cachorro 🐶" if prediction > 0.5 else "Gato 🐱"
    plt.imshow(img)
    plt.title(f"Predição: {label}")
    plt.axis("off")
    plt.show()

